In [2]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core import tools
from langchain.tools import tool
from IPython.display import Markdown


load_dotenv("./.env")

llm = ChatOpenAI(model="gpt-4o")



In [3]:
@tool 
def write_email(to: str, subject:str, content: str) -> str:
    """Write and send email"""
    return f"Email sent to {to} with subject {subject} and content : {content}"

model_with_tools = llm.bind_tools( [write_email] , parallel_tool_calls=False)
output = model_with_tools.invoke("Envia un email a manuonda@gmail.com diciendo que es un crack")

print(output.tool_calls[0])
print(f"Name tool : {output.tool_calls[0]['name']}")

type(output)
args = output.tool_calls[0]['args']
print(f"Args : {args}")

#call the tool
result = write_email.invoke(args)
Markdown(result)


{'name': 'write_email', 'args': {'to': 'manuonda@gmail.com', 'subject': '¡Eres un crack!', 'content': 'Hola Manu,\n\nSolo quería tomarme un momento para decirte que eres un verdadero crack en todo lo que haces. ¡Sigue así!\n\nSaludos,\n\n[Tu Nombre]'}, 'id': 'call_1wwrKiTskbvn9G6RWT983IEq', 'type': 'tool_call'}
Name tool : write_email
Args : {'to': 'manuonda@gmail.com', 'subject': '¡Eres un crack!', 'content': 'Hola Manu,\n\nSolo quería tomarme un momento para decirte que eres un verdadero crack en todo lo que haces. ¡Sigue así!\n\nSaludos,\n\n[Tu Nombre]'}


Email sent to manuonda@gmail.com with subject ¡Eres un crack! and content : Hola Manu,

Solo quería tomarme un momento para decirte que eres un verdadero crack en todo lo que haces. ¡Sigue así!

Saludos,

[Tu Nombre]

In [4]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END

class StateSchema(TypedDict):
    request: str
    email: str

workflow = StateGraph(StateSchema)




def write_email_node(state: StateSchema) -> StateSchema:
    # Imperative code that processes the request
    output = model_with_tools.invoke(state["request"])
    print(output)
    if output.tool_calls:
      args = output.tool_calls[0]['args']
      email = write_email.invoke(args)
      return {"email": email}
    else:
        return {"email": output.content}



workflow = StateGraph(StateSchema)
workflow.add_node("write_email_node", write_email_node)
workflow.add_edge(START, "write_email_node")
workflow.add_edge("write_email_node", END)

app = workflow.compile()

app.invoke({"request": "Draft a response to my boss (boss@company.ai) about tomorrow's meeting"})#


content='Could you please provide more details about the content or purpose of the meeting so I can draft a more specific response?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 66, 'total_tokens': 90, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-6682b948-cad5-4c0b-ba92-dbaa3ca11372-0' usage_metadata={'input_tokens': 66, 'output_tokens': 24, 'total_tokens': 90, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


{'request': "Draft a response to my boss (boss@company.ai) about tomorrow's meeting",
 'email': 'Could you please provide more details about the content or purpose of the meeting so I can draft a more specific response?'}

In [5]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(
    model=llm,
    tools=[write_email],
    prompt="Respond to the user's request using the tools provided"
)
result = agent.invoke(
    {"messages": [{"role": "user", "content": "Draft a response to my boss (boss@company.ai) confirming that I want to attend Interrupt!"}]}
 )
for  m in result["messages"]:
    m.pretty_print()
    

================================ Human Message =================================

Draft a response to my boss (boss@company.ai) confirming that I want to attend Interrupt!
================================== Ai Message ==================================
Tool Calls:
  write_email (call_xeSgtxMaUMU9MDSOlgl5qa8Y)
 Call ID: call_xeSgtxMaUMU9MDSOlgl5qa8Y
  Args:
    to: boss@company.ai
    subject: Confirmation: Attendance at Interrupt! Conference
    content: Hi Boss,

I hope this message finds you well. I am writing to confirm my interest in attending the Interrupt! conference. I believe it will be a great opportunity for professional development and networking.

Please let me know if there are any further steps needed from my end.

Thank you for supporting my participation.

Best regards,

[Your Name]
================================= Tool Message =================================
Name: write_email

Email sent to boss@company.ai with subject Confirmation: Attendance at Interrupt! Conferen

#Persistence 
Threads

It can be very useful to allow agents to pause during long running tasks.

LangGraph has a built-in persistence layer, implemented through checkpointers, to enable this.

When you compile graph with a checkpointer, the checkpointer saves a checkpoint of the graph state at every step.

Checkpoints are saved to a thread, which can be accessed after graph execution completes.

In [14]:
from langgraph.checkpoint.memory import MemorySaver

agent = create_react_agent(
    tools=[write_email],
    model = llm,
    prompt="Responde to the user's request using the tools provided.",
    checkpointer=MemorySaver()
)

config = {"configurable": {"thread_id":"1"}}
result = agent.invoke({"messages":[{"role": "user", "content": "What are somge doog practices for writing emails ? "}]}, config)

#Get th latest state snapshot 
config = {"configurable": {"thread_id": "1"}}
state = agent.get_state(config)
for m in state.values["messages"]:
   m.pretty_print()


================================ Human Message =================================

What are somge doog practices for writing emails ? 
================================== Ai Message ==================================

Here are some good practices for writing emails:

1. **Clear Subject Line**: Use a concise and relevant subject line to give the recipient an idea of the email's content.

2. **Professional Greeting**: Start with a polite greeting. Use the recipient's name if you know it.

3. **Be Concise and Focused**: Keep your email brief and to the point. Focus on one main topic per email.

4. **Use a Professional Tone**: Maintain a formal tone, especially in business communications.

5. **Proper Structure**: Use paragraphs to separate different ideas or topics. This makes your email easier to read.

6. **Use Bullets or Numbers**: If you have multiple points, use bullets or numbers to list them clearly.

7. **Check Spelling and Grammar**: Proofread your email to correct any spelling or 

In [ ]:
#continue the conversation
result = agent.invoke({"messages":["role":"user", "content"]})